In [10]:
import pandas as pd
import re
import string

In [13]:
real_news = pd.read_csv("News_Dataset/True.csv")
fake_news = pd.read_csv("News_Dataset/Fake.csv")
real_news2 = pd.read_csv("addtional_training_data/Real2.csv")
fake_news2 = pd.read_csv("addtional_training_data/Fake2.csv")

In [14]:
real_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [15]:
real_news2.head()

,id,news_url,title,tweet_ids
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,967132259869487105\t967164368768196609\t967215...
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,942953459\t8980098198\t16253717352\t1668513250...
2,politifact333,https://web.archive.org/web/20080204072132/htt...,"Romney makes pitch, hoping to close deal : Ele...",NaN
3,politifact4358,https://web.archive.org/web/20110811143753/htt...,Democratic Leaders Say House Democrats Are Uni...,NaN
4,politifact779,https://web.archive.org/web/20070820164107/htt...,"Budget of the United States Government, FY 2008",89804710374154240\t91270460595109888\t96039619...


In [4]:
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [17]:
fake_news2.head()

,id,news_url,title,tweet_ids
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...


In [20]:
real_news["label"] = 1
fake_news["label"] = 0
real_news2["label"] = 1
fake_news2["label"] = 0

In [21]:
real_news.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [22]:
real_news_model = real_news[["title", "label"]]
fake_news_model = fake_news[["title", "label"]]
real_news_model2 = real_news2[["title", "label"]]
fake_news_model2 = fake_news2[["title", "label"]]

In [23]:
complete_df = pd.concat([real_news_model, fake_news_model, real_news_model2, fake_news_model2], axis=0)
complete_df = complete_df.sample(frac=1, random_state=42).reset_index(drop=True)
complete_df.head()

,title,label
0,German Greens want last nuclear weapons withdr...,1
1,COMEDY GOLD ON DETROIT NEWS: “Willy” Dumps His...,0
2,Trump will 'do everything' to avoid nuclear wa...,1
3,ALT-LEFT PLANS TO HIJACK PRESIDENT TRUMP’S AZ ...,0
4,Forty-four Venezuelan activists released from ...,1


In [24]:
complete_df = complete_df.dropna()

In [25]:
complete_df.isnull().sum()

title    0
label    0
dtype: int64

In [27]:
complete_df = complete_df.drop_duplicates(subset=["title"], keep="first")
complete_df.shape

(39712, 2)

In [28]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans("","", string.punctuation))
    text = text.strip()
    return text

complete_df["title"] = complete_df["title"].apply(clean_text)

complete_df.head()

,title,label
0,german greens want last nuclear weapons withdr...,1
1,comedy gold on detroit news “willy” dumps his ...,0
2,trump will do everything to avoid nuclear war ...,1
3,altleft plans to hijack president trump’s az r...,0
4,fortyfour venezuelan activists released from p...,1


In [29]:
complete_df.to_csv("cleaned_news_headlines.csv", index=False)